In [1]:
data = {
    "M1": {"Region": "Region_1", "Oil_market": 9, "Delivery_points": 11, "Spirit_market": 34, "Growth_category": "A"},
    "M2": {"Region": "Region_1", "Oil_market": 13, "Delivery_points": 47, "Spirit_market": 411, "Growth_category": "A"},
    "M3": {"Region": "Region_1", "Oil_market": 14, "Delivery_points": 44, "Spirit_market": 82, "Growth_category": "A"},
    "M4": {"Region": "Region_1", "Oil_market": 17, "Delivery_points": 25, "Spirit_market": 157, "Growth_category": "B"},
    "M5": {"Region": "Region_1", "Oil_market": 18, "Delivery_points": 10, "Spirit_market": 5, "Growth_category": "A"},
    "M6": {"Region": "Region_1", "Oil_market": 19, "Delivery_points": 26, "Spirit_market": 183, "Growth_category": "A"},
    "M7": {"Region": "Region_1", "Oil_market": 23, "Delivery_points": 26, "Spirit_market": 14, "Growth_category": "B"},
    "M8": {"Region": "Region_1", "Oil_market": 21, "Delivery_points": 54, "Spirit_market": 215, "Growth_category": "B"},
    "M9": {"Region": "Region_2", "Oil_market": 9, "Delivery_points": 18, "Spirit_market": 102, "Growth_category": "B"},
    "M10": {"Region": "Region_2", "Oil_market": 11, "Delivery_points": 51, "Spirit_market": 21, "Growth_category": "A"},
    "M11": {"Region": "Region_2", "Oil_market": 17, "Delivery_points": 20, "Spirit_market": 54, "Growth_category": "B"},
    "M12": {"Region": "Region_2", "Oil_market": 18, "Delivery_points": 105, "Spirit_market": 0, "Growth_category": "B"},
    "M13": {"Region": "Region_2", "Oil_market": 18, "Delivery_points": 7, "Spirit_market": 6, "Growth_category": "B"},
    "M14": {"Region": "Region_2", "Oil_market": 17, "Delivery_points": 16, "Spirit_market": 96, "Growth_category": "B"},
    "M15": {"Region": "Region_2", "Oil_market": 22, "Delivery_points": 34, "Spirit_market": 118, "Growth_category": "A"},
    "M16": {"Region": "Region_2", "Oil_market": 24, "Delivery_points": 100, "Spirit_market": 112, "Growth_category": "B"},
    "M17": {"Region": "Region_2", "Oil_market": 36, "Delivery_points": 50, "Spirit_market": 535, "Growth_category": "B"},
    "M18": {"Region": "Region_2", "Oil_market": 43, "Delivery_points": 21, "Spirit_market": 8, "Growth_category": "B"},
    "M19": {"Region": "Region_3", "Oil_market": 6, "Delivery_points": 11, "Spirit_market": 53, "Growth_category": "B"},
    "M20": {"Region": "Region_3", "Oil_market": 15, "Delivery_points": 19, "Spirit_market": 28, "Growth_category": "A"},
    "M21": {"Region": "Region_3", "Oil_market": 15, "Delivery_points": 14, "Spirit_market": 69, "Growth_category": "B"},
    "M22": {"Region": "Region_3", "Oil_market": 25, "Delivery_points": 10, "Spirit_market": 65, "Growth_category": "B"},
    "M23": {"Region": "Region_3", "Oil_market": 39, "Delivery_points": 11, "Spirit_market": 27, "Growth_category": "B"}
}

retailers = list(data.keys())
regions = ['Region_1', 'Region_2', 'Region_3']
categories = ['A', 'B']

# Calculate total values
total_delivery = sum(data[retailer]["Delivery_points"] for retailer in retailers)
total_spirit = sum(data[retailer]["Spirit_market"] for retailer in retailers)
total_oil = {region: sum(data[retailer]["Oil_market"] for retailer in retailers if data[retailer]["Region"] == region) for region in regions}
total_retailers = {category: sum(1 for retailer in retailers if data[retailer]["Growth_category"] == category) for category in categories}


In [2]:
from ortools.linear_solver import pywraplp

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define the decision variables
var = {}
for retailer in retailers:
    var[f'c_{retailer}_D1'] = solver.BoolVar(f'c_{retailer}_D1')
    # var[f'c_{retailer}_D2'] = solver.BoolVar(f'c_{retailer}_D2')

# Define deviation variables
var['delivery_break'] = solver.NumVar(0, 0.05, 'delivery_break')
var['spirit_break'] = solver.NumVar(0, 0.05, 'spirit_break')
for region in regions:
    var[f'oil_{region}_break'] = solver.NumVar(0, 0.05, f'oil_{region}_break')
for category in categories:
    var[f'group_break_{category}'] = solver.NumVar(0, 0.05, f'group_break_{category}')

# # Assignment Constraints
# for retailer in retailers:
#     solver.Add(var[f'c_{retailer}_D1'] + var[f'c_{retailer}_D2'] == 1)

# Splitting Constraints
# Total Number of Delivery Points Splitting
delivery_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Delivery_points"] for retailer in retailers)
solver.Add(delivery_d1 <= 0.45 * total_delivery)
solver.Add(delivery_d1 >= 0.35 * total_delivery)
solver.Add(var['delivery_break'] >= (delivery_d1 / total_delivery) - 0.40)
solver.Add(var['delivery_break'] >= -(delivery_d1 / total_delivery) + 0.40)

# Spirit Market Splitting
spirit_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Spirit_market"] for retailer in retailers)
solver.Add(spirit_d1 <= 0.45 * total_spirit)
solver.Add(spirit_d1 >= 0.35 * total_spirit)
solver.Add(var['spirit_break'] >= (spirit_d1 / total_spirit) - 0.40)
solver.Add(var['spirit_break'] >= -(spirit_d1 / total_spirit) + 0.40)

# Oil Market Splitting by Region
for region in regions:
    oil_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Oil_market"] for retailer in retailers if data[retailer]["Region"] == region)
    solver.Add(oil_d1 <= 0.45 * total_oil[region])
    solver.Add(oil_d1 >= 0.35 * total_oil[region])
    solver.Add(var[f'oil_{region}_break'] >= (oil_d1 / total_oil[region]) - 0.40)
    solver.Add(var[f'oil_{region}_break'] >= -(oil_d1 / total_oil[region]) + 0.40)

# Retailer Splitting by Growth Category
for category in categories:
    n_retailer_d1 = sum(var[f'c_{retailer}_D1'] for retailer in retailers if data[retailer]['Growth_category'] == category)
    solver.Add(n_retailer_d1 <= 0.45 * total_retailers[category])
    solver.Add(n_retailer_d1 >= 0.35 * total_retailers[category])
    solver.Add(var[f'group_break_{category}'] >= (n_retailer_d1 / total_retailers[category]) - 0.40)
    solver.Add(var[f'group_break_{category}'] >= -(n_retailer_d1 / total_retailers[category]) + 0.40)

# Objective function
total_deviation = solver.Objective()
total_deviation.SetCoefficient(var['delivery_break'], 1)
total_deviation.SetCoefficient(var['spirit_break'], 1)
for region in regions:
    total_deviation.SetCoefficient(var[f'oil_{region}_break'], 1)
for category in categories:
    total_deviation.SetCoefficient(var[f'group_break_{category}'], 1)
total_deviation.SetMinimization()

# Solve the problem
status = solver.Solve()

# Print the results
if status == pywraplp.Solver.OPTIMAL:
    print('The Optimal Deviation is:', total_deviation.Value())
    for retailer in retailers:
        if var[f'c_{retailer}_D1'].solution_value() == 1:
            print(f'Retailer {retailer} is assigned to Division D1')
        else:
            print(f'Retailer {retailer} is assigned to Division D2')
else:
    print('No Optimal Solution found.')


The Optimal Deviation is: 0.05297617969288393
Retailer M1 is assigned to Division D1
Retailer M2 is assigned to Division D1
Retailer M3 is assigned to Division D1
Retailer M4 is assigned to Division D1
Retailer M5 is assigned to Division D2
Retailer M6 is assigned to Division D2
Retailer M7 is assigned to Division D2
Retailer M8 is assigned to Division D2
Retailer M9 is assigned to Division D2
Retailer M10 is assigned to Division D2
Retailer M11 is assigned to Division D1
Retailer M12 is assigned to Division D2
Retailer M13 is assigned to Division D2
Retailer M14 is assigned to Division D2
Retailer M15 is assigned to Division D2
Retailer M16 is assigned to Division D1
Retailer M17 is assigned to Division D2
Retailer M18 is assigned to Division D1
Retailer M19 is assigned to Division D2
Retailer M20 is assigned to Division D2
Retailer M21 is assigned to Division D1
Retailer M22 is assigned to Division D1
Retailer M23 is assigned to Division D2


In [4]:
from ortools.linear_solver import pywraplp

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Define the decision variables
var = {}
for retailer in retailers:
    var[f'c_{retailer}_D1'] = solver.BoolVar(f'c_{retailer}_D1')
    # var[f'c_{retailer}_D2'] = solver.BoolVar(f'c_{retailer}_D2')

# Define deviation variables
var['delivery_break'] = solver.NumVar(0, 0.05, 'delivery_break')
var['spirit_break'] = solver.NumVar(0, 0.05, 'spirit_break')
for region in regions:
    var[f'oil_{region}_break'] = solver.NumVar(0, 0.05, f'oil_{region}_break')
for category in categories:
    var[f'group_break_{category}'] = solver.NumVar(0, 0.05, f'group_break_{category}')
var['max'] = solver.NumVar(0, 0.05, 'maximum')
# # Assignment Constraints
# for retailer in retailers:
#     solver.Add(var[f'c_{retailer}_D1'] + var[f'c_{retailer}_D2'] == 1)

# Splitting Constraints
# Total Number of Delivery Points Splitting
delivery_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Delivery_points"] for retailer in retailers)
solver.Add(delivery_d1 <= 0.45 * total_delivery)
solver.Add(delivery_d1 >= 0.35 * total_delivery)
solver.Add(var['delivery_break'] >= (delivery_d1 / total_delivery) - 0.40)
solver.Add(var['delivery_break'] >= -(delivery_d1 / total_delivery) + 0.40)

# Spirit Market Splitting
spirit_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Spirit_market"] for retailer in retailers)
solver.Add(spirit_d1 <= 0.45 * total_spirit)
solver.Add(spirit_d1 >= 0.35 * total_spirit)
solver.Add(var['spirit_break'] >= (spirit_d1 / total_spirit) - 0.40)
solver.Add(var['spirit_break'] >= -(spirit_d1 / total_spirit) + 0.40)

# Oil Market Splitting by Region
for region in regions:
    oil_d1 = sum(var[f'c_{retailer}_D1'] * data[retailer]["Oil_market"] for retailer in retailers if data[retailer]["Region"] == region)
    solver.Add(oil_d1 <= 0.45 * total_oil[region])
    solver.Add(oil_d1 >= 0.35 * total_oil[region])
    solver.Add(var[f'oil_{region}_break'] >= (oil_d1 / total_oil[region]) - 0.40)
    solver.Add(var[f'oil_{region}_break'] >= -(oil_d1 / total_oil[region]) + 0.40)

# Retailer Splitting by Growth Category
for category in categories:
    n_retailer_d1 = sum(var[f'c_{retailer}_D1'] for retailer in retailers if data[retailer]['Growth_category'] == category)
    solver.Add(n_retailer_d1 <= 0.45 * total_retailers[category])
    solver.Add(n_retailer_d1 >= 0.35 * total_retailers[category])
    solver.Add(var[f'group_break_{category}'] >= (n_retailer_d1 / total_retailers[category]) - 0.40)
    solver.Add(var[f'group_break_{category}'] >= -(n_retailer_d1 / total_retailers[category]) + 0.40)
# EXtra Constrainsts
# Objective function
solver.Add(var['delivery_break'] <= var['max'])
solver.Add(var['spirit_break'] <= var['max'])
for region in regions:
    solver.Add(var[f'oil_{region}_break'] <= var['max'])
for category in categories:
    solver.Add(var[f'group_break_{category}'] <= var['max'])

total_deviation = solver.Objective()
total_deviation.SetCoefficient(var['max'],1)
total_deviation.SetMinimization()

# Solve the problem
status = solver.Solve()

# Print the results
if status == pywraplp.Solver.OPTIMAL:
    print('The Optimal Deviation is:', total_deviation.Value())
    for retailer in retailers:
        if var[f'c_{retailer}_D1'].solution_value() == 1:
            print(f'Retailer {retailer} is assigned to Division D1')
        else:
            print(f'Retailer {retailer} is assigned to Division D2')
else:
    print('No Optimal Solution found.')

The Optimal Deviation is: 0.025000000000000022
Retailer M1 is assigned to Division D2
Retailer M2 is assigned to Division D2
Retailer M3 is assigned to Division D1
Retailer M4 is assigned to Division D2
Retailer M5 is assigned to Division D2
Retailer M6 is assigned to Division D1
Retailer M7 is assigned to Division D1
Retailer M8 is assigned to Division D2
Retailer M9 is assigned to Division D1
Retailer M10 is assigned to Division D2
Retailer M11 is assigned to Division D2
Retailer M12 is assigned to Division D1
Retailer M13 is assigned to Division D1
Retailer M14 is assigned to Division D2
Retailer M15 is assigned to Division D2
Retailer M16 is assigned to Division D2
Retailer M17 is assigned to Division D1
Retailer M18 is assigned to Division D2
Retailer M19 is assigned to Division D2
Retailer M20 is assigned to Division D1
Retailer M21 is assigned to Division D2
Retailer M22 is assigned to Division D1
Retailer M23 is assigned to Division D2


In [1]:
from ortools.linear_solver import pywraplp

# Define the time slots and durations
time_slots = range(5)
durations = [6, 3, 6, 3, 6]

# Define the types of power plants and their availability
types = range(3)
availability = [12, 10, 5]

# Define the power plant data
power_plant_data = {
    0: {
        "Minimum_level_(MW)": 850,
        "Maximum_level_(MW)": 2000,
        "Cost_per_hour_at_minimum": 1000,
        "Cost_per_hour_per_megawatt_above_minimum": 2,
        "Fixed_cost": 2000
    },
    1: {
        "Minimum_level_(MW)": 1250,
        "Maximum_level_(MW)": 1750,
        "Cost_per_hour_at_minimum": 2600,
        "Cost_per_hour_per_megawatt_above_minimum": 1.30,
        "Fixed_cost": 1000
    },
    2: {
        "Minimum_level_(MW)": 1500,
        "Maximum_level_(MW)": 4000,
        "Cost_per_hour_at_minimum": 3000,
        "Cost_per_hour_per_megawatt_above_minimum": 3,
        "Fixed_cost": 500
    }
}

# Define the demand in each time slot (in megawatts)
demands = [15000, 30000, 25000, 40000, 27000] # In megawatts

# Create the solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Decision Variables
var = {}
for t in types:
    var[f'active_{t}'] = [solver.NumVar(0, availability[t], f'active_{t}_{time}') for time in time_slots]
    var[f'started_{t}'] = [solver.NumVar(0, availability[t], f'started_{t}_{time}') for time in time_slots]
    var[f'output_{t}'] = [solver.NumVar(0, solver.infinity(), f'output_{t}_{time}') for time in time_slots]

# Constraints
# Starting Constraint
for time in time_slots:
    if time == 0:
        for type in types:
            solver.Add(var[f'started_{type}'][time] >= var[f'active_{type}'][time])
    else:
        for type in types:
            solver.Add(var[f'started_{type}'][time] >= var[f'active_{type}'][time] - var[f'active_{type}'][time-1])
# Output Constraints
for time in time_slots:
    for t in types:
        solver.Add(var[f'output_{t}'][time] >= var[f'active_{t}'][time] * power_plant_data[t]["Minimum_level_(MW)"])
        solver.Add(var[f'output_{t}'][time] <= var[f'active_{t}'][time] * power_plant_data[t]["Maximum_level_(MW)"])

# Demand Constraints with 15% reserve
for time in time_slots:
    solver.Add(sum(var[f'output_{t}'][time] for t in types) == demands[time])
    solver.Add(sum(var[f'active_{t}'][time] * power_plant_data[t]['Maximum_level_(MW)'] for t in types) >= 1.15 * demands[time])

# Objective
cost = solver.Objective()
for time in time_slots:
    for t in types:
        # Cost per hour at minimum level
        cost.SetCoefficient(var[f'active_{t}'][time], power_plant_data[t]['Cost_per_hour_at_minimum']
                            - power_plant_data[t]['Minimum_level_(MW)'] * power_plant_data[t]['Cost_per_hour_per_megawatt_above_minimum'])
        # Additional cost for output above minimum level
        cost.SetCoefficient(var[f'output_{t}'][time], power_plant_data[t]['Cost_per_hour_per_megawatt_above_minimum'])
        # Fixed startup cost
        cost.SetCoefficient(var[f'started_{t}'][time], power_plant_data[t]['Fixed_cost'])
        

cost.SetMinimization()

# Solve the problem
status = solver.Solve()

# Print the results
if status == pywraplp.Solver.OPTIMAL:
    print('The Optimal Cost is:', cost.Value())
    for time in time_slots:
        print(f"Time slot {time}:")
        for t in types:
            print(f"  Type {t} - Active: {var[f'active_{t}'][time].solution_value()}, Output: {var[f'output_{t}'][time].solution_value()}")
else:
    print('No Optimal Solution found.')


The Optimal Cost is: 256214.28571428574
Time slot 0:
  Type 0 - Active: 12.0, Output: 10200.000000000002
  Type 1 - Active: 2.742857142857143, Output: 4800.0
  Type 2 - Active: 0.0, Output: 0.0
Time slot 1:
  Type 0 - Active: 12.0, Output: 15200.0
  Type 1 - Active: 8.457142857142859, Output: 14800.000000000002
  Type 2 - Active: 0.0, Output: 0.0
Time slot 2:
  Type 0 - Active: 12.0, Output: 10200.000000000002
  Type 1 - Active: 8.457142857142859, Output: 14800.000000000002
  Type 2 - Active: 0.0, Output: 0.0
Time slot 3:
  Type 0 - Active: 12.0, Output: 22499.999999999996
  Type 1 - Active: 8.457142857142859, Output: 14800.000000000002
  Type 2 - Active: 1.8000000000000005, Output: 2700.000000000001
Time slot 4:
  Type 0 - Active: 12.0, Output: 12200.000000000002
  Type 1 - Active: 8.457142857142859, Output: 14800.000000000002
  Type 2 - Active: 0.0, Output: 0.0


In [6]:
data = {}
data['levels'] = range(4)
data['profit'] = {}
data['profit'][0] = [[6.0]]
data['profit'][1] = [[12,6],[5,4]]
data['profit'][2] = [[4,4,2],[3,3,1],[2,2,0.5]]
data['profit'][3] = [[1.5,1.5,1.5,0.75],[1.5,2.0,1.5,0.75],
                     [1,1,0.75,0.5],[0.75,0.75,0.5,0.25]]
data['excavation_cost'] = [10000,8000,6000,3000]
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

# Decision Variables
vars = {}
vars['open'] = [[solver.BoolVar(f'open_{box}_{level}') for box in range(len(data['profit'][level][0])**2)] for level in data['levels']]

# Constraints
# Opening Constraint

for level in range(3):
    add = -2
    for box in range(len(data['profit'][level][0])**2):
        if box % len(data['profit'][level][0]) == 0:
            add += 2
        else:
            add += 1
        solver.Add(4*vars['open'][level][box] <= vars['open'][level+1][add]
                                                + vars['open'][level+1][add + 1]
                                                + vars['open'][level+1][add + 2 + level]
                                                + vars['open'][level+1][add + 3 + level]) 
        
revenue = solver.Objective()
for level in range(4):
    for box in range(len(data['profit'][level][0])**2):
        row = int(box/len(data['profit'][level][0]))
        col = box%len(data['profit'][level][0])
        revenue.SetCoefficient(vars['open'][level][box], data['profit'][level][row][col]*2000 - data['excavation_cost'][level])

revenue.SetMaximization()

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("The maximum revenue achievable is: ",revenue.Value())
else:
    print("Optimal solution does not exist. ")


The maximum revenue achievable is:  17500.0


In [12]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')
colors = ['b', 'w']
diag_types = ['pos', 'neg']

# Decision variables
ball_colour = [[[solver.BoolVar(f'ball_colour_{row}_{col}_{height}') for row in range(3)] for col in range(3)] for height in range(3)]

# Row-wise lines
r_lines = {color: [[solver.BoolVar(f'r_lines_{col}_{height}_{color}') for col in range(3)] for height in range(3)] for color in colors}

# Column-wise lines
c_lines = {color: [[solver.BoolVar(f'c_lines_{row}_{height}_{color}') for row in range(3)] for height in range(3)] for color in colors}

# Height-wise lines
h_lines = {color: [[solver.BoolVar(f'h_lines_{row}_{col}_{color}') for row in range(3)] for col in range(3)] for color in colors}

# Horizontal diagonal constraints
# Row-wise diagonal constraints
r_diag = {color: {type : [solver.BoolVar(f'r_diag_{row}_{color}_{type}') for row in range(3)] for type in diag_types} for color in colors}

# Column-wise diagonal constraints
c_diag = {color: {type : [solver.BoolVar(f'c_diag_{col}_{color}_{type}') for col in range(3)] for type in diag_types} for color in colors}

# Height-wise diagonal constraints
h_diag = {color: {type : [solver.BoolVar(f'h_diag_{height}_{color}_{type}') for height in range(3)] for type in diag_types} for color in colors}

# 3D diagonal lines
diag_3d = {color: [solver.BoolVar(f'diag3d_{i}_{color}') for i in range(4)] for color in colors}

# Constraints
# Colour constraints
white_ball_count = sum(ball_colour[height][col][row] for height in range(3) for col in range(3) for row in range(3))
solver.Add(white_ball_count == 13)
# Row-wise constraints
for col in range(3):
    for height in range(3):
        solver.Add(3 * r_lines['w'][height][col] <= sum([ball_colour[height][col][row] for row in range(3)]))  # for same white balls
        solver.Add(3 * r_lines['w'][height][col] + 2 >= sum([ball_colour[height][col][row] for row in range(3)]))
        solver.Add(3 * r_lines['b'][height][col] <= 3 - sum([ball_colour[height][col][row] for row in range(3)]))  # for same black balls
        solver.Add(3 * r_lines['b'][height][col] + 2 >= 3 - sum([ball_colour[height][col][row] for row in range(3)]))

# Column-wise constraints
for row in range(3):
    for height in range(3):
        solver.Add(3 * c_lines['w'][height][row] <= sum([ball_colour[height][col][row] for col in range(3)]))  # for same white balls
        solver.Add(3 * c_lines['w'][height][row] + 2 >= sum([ball_colour[height][col][row] for col in range(3)]))
        solver.Add(3 * c_lines['b'][height][row] <= 3 - sum([ball_colour[height][col][row] for col in range(3)]))  # for same black balls
        solver.Add(3 * c_lines['b'][height][row] + 2 >= 3 - sum([ball_colour[height][col][row] for col in range(3)]))

# Height-wise constraints
for row in range(3):
    for col in range(3):
        solver.Add(3 * h_lines['w'][col][row] <= sum([ball_colour[height][col][row] for height in range(3)]))  # for same white balls
        solver.Add(3 * h_lines['w'][col][row] + 2 >= sum([ball_colour[height][col][row] for height in range(3)]))
        solver.Add(3 * h_lines['b'][col][row] <= 3 - sum([ball_colour[height][col][row] for height in range(3)]))  # for same black balls
        solver.Add(3 * h_lines['b'][col][row] + 2 >= 3 - sum([ball_colour[height][col][row] for height in range(3)]))

# Diagonal Constraints
# Row-wise diagonal
for row in range(3):
    for type in diag_types:
        if type == 'pos':
            height = 0
            col = 0
            diag_sum = sum(ball_colour[height + i][col + i][row] for i in range(3))
        else:
            height = 2
            col = 0
            diag_sum = sum(ball_colour[height - i][col + i][row] for i in range(3))
        solver.Add(3 * r_diag['w'][type][row] <= diag_sum)
        solver.Add(3 * r_diag['w'][type][row] + 2 >= diag_sum)
        solver.Add(3 * r_diag['b'][type][row] <= 3 - diag_sum)
        solver.Add(3 * r_diag['b'][type][row] + 2 >= 3 - diag_sum)

# Column-wise diagonal
for col in range(3):
    for type in diag_types:
        if type == 'pos':
            height = 0
            row = 0
            diag_sum = sum(ball_colour[height + i][col][row + i] for i in range(3))
        else:
            height = 2
            row = 0
            diag_sum = sum(ball_colour[height - i][col][row + i] for i in range(3))
        solver.Add(3 * c_diag['w'][type][col] <= diag_sum)
        solver.Add(3 * c_diag['w'][type][col] + 2 >= diag_sum)
        solver.Add(3 * c_diag['b'][type][col] <= 3 - diag_sum)
        solver.Add(3 * c_diag['b'][type][col] + 2 >= 3 - diag_sum)

# Height-wise diagonal
for height in range(3):
    for type in diag_types:
        if type == 'pos':
            row = 0
            col = 0
            diag_sum = sum(ball_colour[height][col + i][row + i] for i in range(3))
        else:
            row = 2
            col = 0
            diag_sum = sum(ball_colour[height][col + i][row - i] for i in range(3))
        solver.Add(3 * h_diag['w'][type][height] <= diag_sum)
        solver.Add(3 * h_diag['w'][type][height] + 2 >= diag_sum)
        solver.Add(3 * h_diag['b'][type][height] <= 3 - diag_sum)
        solver.Add(3 * h_diag['b'][type][height] + 2 >= 3 - diag_sum)

# 3D diagonal
mul = [(1, 1), (1, -1), (-1, 1), (-1, -1)]
start = [(0, 0), (0, 2), (2, 0), (2, 2)]
height = 0
for diag in range(4):
    (col, row) = start[diag]
    diag_sum = sum([ball_colour[height + i][col + i * mul[diag][0]][row + i * mul[diag][1]] for i in range(3)])
    solver.Add(3 * diag_3d['w'][diag] <= diag_sum)
    solver.Add(3 * diag_3d['w'][diag] + 2 >= diag_sum)
    solver.Add(3 * diag_3d['b'][diag] <= 3 - diag_sum)
    solver.Add(3 * diag_3d['b'][diag] + 2 >= 3 - diag_sum)

# Objective function
objective = solver.Objective()
# Add coefficients for row-wise lines
for color in colors:
    for height in range(3):
        for col in range(3):
            objective.SetCoefficient(r_lines[color][height][col], 1)

# Add coefficients for column-wise lines
for color in colors:
    for height in range(3):
        for row in range(3):
            objective.SetCoefficient(c_lines[color][height][row], 1)

# Add coefficients for height-wise lines
for color in colors:
    for col in range(3):
        for row in range(3):
            objective.SetCoefficient(h_lines[color][col][row], 1)

# Add coefficients for row-wise diagonals
for color in colors:
    for type in diag_types:
        for row in range(3):
            objective.SetCoefficient(r_diag[color][type][row], 1)

# Add coefficients for column-wise diagonals
for color in colors:
    for type in diag_types:
        for col in range(3):
            objective.SetCoefficient(c_diag[color][type][col], 1)

# Add coefficients for height-wise diagonals
for color in colors:
    for type in diag_types:
        for height in range(3):
            objective.SetCoefficient(h_diag[color][type][height], 1)

# Add coefficients for 3D diagonals
for color in colors:
    for diag in range(4):
        objective.SetCoefficient(diag_3d[color][diag], 1)

objective.SetMinimization()

# Solve the problem
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('The least poosible number of lines are: ',objective.Value())
    for height in range(3):
        for row in range(3):
            for col in range(3):
                print(f'ball_colour[{height}][{col}][{row}] = {ball_colour[height][col][row].solution_value()}')
else:
    print('The problem does not have an optimal solution.')


Solution:
The least poosible number of lines are:  3.9999999999999982
ball_colour[0][0][0] = 1.0
ball_colour[0][1][0] = 0.0
ball_colour[0][2][0] = 1.0
ball_colour[0][0][1] = 1.0
ball_colour[0][1][1] = 1.0
ball_colour[0][2][1] = 0.0
ball_colour[0][0][2] = 0.0
ball_colour[0][1][2] = 1.0
ball_colour[0][2][2] = 0.0
ball_colour[1][0][0] = 0.0
ball_colour[1][1][0] = 0.0
ball_colour[1][2][0] = 0.0
ball_colour[1][0][1] = 0.0
ball_colour[1][1][1] = 0.0
ball_colour[1][2][1] = 1.0
ball_colour[1][0][2] = 1.0
ball_colour[1][1][2] = 1.0
ball_colour[1][2][2] = 1.0
ball_colour[2][0][0] = 1.0
ball_colour[2][1][0] = 0.0
ball_colour[2][2][0] = 1.0
ball_colour[2][0][1] = 1.0
ball_colour[2][1][1] = 0.0
ball_colour[2][2][1] = 0.0
ball_colour[2][0][2] = 0.0
ball_colour[2][1][2] = 1.0
ball_colour[2][2][2] = 0.0


In [3]:
import itertools
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('SCIP')

var_combs = list(itertools.product([0,1], repeat = 8))
old_coeff = [9,13,-14,17,13,-19,23,21]
old_rhs = 37

# Decision Variables
z = [solver.BoolVar( f'z_{i}') for i in range(len(var_combs))]
new_coeff = [solver.NumVar(-solver.infinity(), solver.infinity(), f'coeff_{index}') for index in range(len(old_coeff))]
new_rhs = solver.NumVar(-solver.infinity(), solver.infinity(), "rhs")
obj = solver.NumVar(0, solver.infinity(), 'abs_rhs')

# Constraints
# Same sign constraints
for i in range(len(new_coeff)):
    solver.Add(new_coeff[i]*old_coeff[i] >= 0)

for i, var in enumerate(var_combs):
    solver.Add(sum(old_coeff[j]*var[j] for j in range(len(var))) <= old_rhs + solver.infinity()*(1- z[i]))
    solver.Add(sum(old_coeff[j]*var[j] for j in range(len(var))) >= old_rhs - solver.infinity()*z[i])
    solver.Add(sum(new_coeff[j]*var[j] for j in range(len(var))) <= new_rhs + solver.infinity()*(1 - z[i]))
    solver.Add(sum(new_coeff[j]*var[j] for j in range(len(var))) >= new_rhs - solver.infinity()*z[i])
 

solver.Add(obj >= new_rhs)
solver.Add(-obj <= new_rhs)

# Objective
objective = solver.Objective()

objective.SetCoefficient(obj, 1)

objective.SetMinimization()

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("The final objective is: ", objective.Value())
else:
    print("No Optimal Solution exists.")


No Optimal Solution exists.
